# Zillow Exercises

## Acquire and Summarize

### 1. Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [16]:
from imports import *
# set columnns display format
pd.set_option('display.max_columns', None)
# default pandas decimal number display format
pd.options.display.float_format = '{:20,.3f}'.format

In [2]:
# from our acquire.py:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
def new_zillow_data():
    '''
    This function reads the Zillow data from the mySQL database into a df.
    '''
    # Create SQL query.
    sql_query = '''
    SELECT 
    prop.*,
    pred.logerror,
    pred.transactiondate,
    air.airconditioningdesc,
    arch.architecturalstyledesc,
    build.buildingclassdesc,
    heat.heatingorsystemdesc,
    landuse.propertylandusedesc,
    story.storydesc,
    construct.typeconstructiondesc
FROM
    properties_2017 prop
        INNER JOIN
    (SELECT 
        parcelid, logerror, MAX(transactiondate) AS transactiondate
    FROM
        predictions_2017
    GROUP BY parcelid , logerror) pred USING (parcelid)
        LEFT JOIN
    airconditioningtype air USING (airconditioningtypeid)
        LEFT JOIN
    architecturalstyletype arch USING (architecturalstyletypeid)
        LEFT JOIN
    buildingclasstype build USING (buildingclasstypeid)
        LEFT JOIN
    heatingorsystemtype heat USING (heatingorsystemtypeid)
        LEFT JOIN
    propertylandusetype landuse USING (propertylandusetypeid)
        LEFT JOIN
    storytype story USING (storytypeid)
        LEFT JOIN
    typeconstructiontype construct USING (typeconstructiontypeid)
WHERE
    prop.latitude IS NOT NULL
        AND prop.longitude IS NOT NULL
        AND transactiondate <= '2017-12-31';
    '''
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    df = df.drop(columns='id')
    return df

def get_zillow_data():
    '''
    This function reads in zillow data from Zillow database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('zillow_data.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('zillow_data.csv', index_col=0)
        df = df.drop(columns='id')
    else:
        
        # Read fresh data from db into a DataFrame.
        df = new_zillow_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('zillow_data.csv')
        
    return df

In [3]:
df = get_zillow_data()

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [4]:
df.shape

(77574, 67)

In [5]:
df.columns

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

In [8]:
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,261.0,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,261.0,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,1.0,010C,266.0,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [9]:
df.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,7.757400e+04,25006.000000,206.000000,50.000000,77574.000000,77574.000000,15.000000,49808.000000,76959.000000,614.0,6035.000000,77374.000000,73919.000000,42.000000,3027.000000,6035.000000,386.000000,77574.000000,8287.000000,76959.000000,25517.000000,25517.000000,1539.0,49569.000000,7.757400e+04,7.757400e+04,6.931700e+04,16173.0,869.000000,465.0,1074.0,15078.0,77574.000000,7.757400e+04,76102.000000,77574.000000,30973.000000,77524.000000,77574.000000,50.0,10105.000000,222.000000,50702.000000,2393.000000,70.000000,77305.000000,17596.000000,172.0,7.745900e+04,7.757300e+04,77574.0,7.757200e+04,77569.000000,2900.000000,7.732700e+04,77574.000000
mean,1.300630e+07,1.812045,7.388350,679.720000,2.298521,3.053252,3.933333,6.533830,2.316389,66.0,1366.198012,1784.948497,1760.296906,1388.761905,2354.491245,1381.426678,2082.500000,6048.812721,1.191263,2.250107,1.815339,350.055845,1.0,3.921645,3.400837e+07,-1.182037e+08,2.997480e+04,1.0,517.930955,1.0,1.0,1.0,261.824477,6.049123e+07,33682.279598,2534.528631,187730.297162,96586.678306,1.476139,7.0,1.009599,6.040541,1.110311,305.460928,216.385714,1968.611047,1.434246,1.0,1.892829e+05,4.901477e+05,2016.0,3.011468e+05,5995.855691,14.088276,6.049666e+13,0.016805
std,3.478043e+06,2.965823,2.734542,689.703546,0.996706,1.140447,0.258199,1.722041,0.979690,0.0,670.803630,954.265049,934.367511,122.220874,1186.985442,725.904022,1240.382784,20.745399,0.491031,0.966485,0.588329,261.651602,0.0,3.594814,2.652895e+05,3.593833e+05,1.233085e+05,0.0,156.569664,0.0,0.0,0.0,5.141733,2.058777e+05,47212.172270,801.452437,165045.226146,3793.673202,2.823613,0.0,0.118578,0.557285,1.169978,238.735241,190.177514,23.793331,0.544518,0.0,2.304149e+05,6.538059e+05,0.0,4.927317e+05,7628.912331,2.181281,1.533386e+12,0.170742
min,1.071186e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,66.0,44.000000,128.000000,128.000000,1056.000000,598.000000,44.000000,380.000000,6037.000000,1.000000,1.000000,0.000000,0.000000,1.0,1.000000,3.333953e+07,-1.194754e+08,2.360000e+02,1.0,24.000000,1.0,1.0,1.0,31.000000,6.037101e+07,3491.000000,1286.000000,6952.000000,95982.000000,0.000000,7.0,1.000000,4.000000,1.000000,11.000000,12.000000,1824.000000,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,1.153819e+07,1.000000,7.000000,273.000000,2.000000,2.000000,4.000000,6.000000,2.000000,66.0,955.000000,1182.000000,1172.000000,1344.000000,1624.000000,956.000000,993.750000,6037.000000,1.000000,2.000000,2.000000,0.000000,1.0,2.000000,3.381463e+07,-1.184150e+08,5.700000e+03,1.0,424.000000,1.0,1.0,1.0,261.000000,6.037311e+07,12447.000000,1286.000000,46736.000000,96193.000000,0.000000,7.0,1.000000,6.000000,1.000000,170.000000,61.500000,1953.000000,1.000000,1.0,8.418200e+04,2.068980e+05,2016.0,8.529200e+04,2712.610000,14.000000,6.037311e+13,-0.024310
50%,1.253050e+07,1.000000,7.000000,515.000000,2.000000,3.000000,4.000000,6.000000,2.000000,66.0,1257.000000,1542.000000,1523.000000,1440.000000,2088.000000,1259.000000,1812.500000,6037.000000,1.000000,2.000000,2.000000,436.000000,1.0

In [11]:
print(f'rows: {df.shape[0]}')
print(f'columns: {df.shape[1]}')

rows: 77574
columns: 67


#### Nulls by Columns

In [12]:
df.isnull().head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,False,True,True,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
1,False,True,True,True,False,False,True,True,False,True,False,False,False,True,True,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
2,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,False,True,True,True,False,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
3,False,True,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,True,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,True,True,True,False,False,True,True
4,False,False,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,True


In [17]:
# Percentage of nulls in each column
df.isnull().sum()/df.shape[0]*100

parcelid                                      0.000
airconditioningtypeid                        67.765
architecturalstyletypeid                     99.734
basementsqft                                 99.936
bathroomcnt                                   0.000
bedroomcnt                                    0.000
buildingclasstypeid                          99.981
buildingqualitytypeid                        35.793
calculatedbathnbr                             0.793
decktypeid                                   99.208
finishedfloor1squarefeet                     92.220
calculatedfinishedsquarefeet                  0.258
finishedsquarefeet12                          4.712
finishedsquarefeet13                         99.946
finishedsquarefeet15                         96.098
finishedsquarefeet50                         92.220
finishedsquarefeet6                          99.502
fips                                          0.000
fireplacecnt                                 89.317
fullbathcnt 

In [18]:
nulls_col = pd.DataFrame({'num_rows_missing': df.isnull().sum(), 'pct_rows_missing': (df.isnull().sum()/df.shape[0]*100)})
nulls_col.sort_values(by = 'num_rows_missing', ascending = False).head(10)

,num_rows_missing,pct_rows_missing
buildingclasstypeid,77559,99.981
buildingclassdesc,77559,99.981
finishedsquarefeet13,77532,99.946
basementsqft,77524,99.936
storydesc,77524,99.936
storytypeid,77524,99.936
yardbuildingsqft26,77504,99.910
fireplaceflag,77402,99.778
architecturalstyletypeid,77368,99.734
architecturalstyledesc,77368,99.734


### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [45]:
num_rows_missing = df.isnull().sum()
rows = df.shape[0]
pct_miss = num_rows_missing / rows * 100
cols_missing = pd.DataFrame({'num_rows_missing': num_rows_missing, 'percent_rows_missing': pct_miss})
cols_missing[cols_missing.percent_rows_missing <= 0.6].index
df[cols_missing[cols_missing.percent_rows_missing <= 0.6].index]

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,propertylandusedesc
0,14297519,3.500,4.000,"3,100.000","6,059.000","33,634,931.000","-117,869,207.000",122,261.000,"60,590,630.072","1,286.000","96,978.000",0.000,"1,998.000","485,713.000","1,023,282.000","2,016.000","537,569.000","11,013.720","60,590,630,072,012.000",0.026,2017-01-01,Single Family Residential
1,17052889,1.000,2.000,"1,465.000","6,111.000","34,449,266.000","-119,281,531.000",1110,261.000,"61,110,010.023","2,061.000","97,099.000",5.000,"1,967.000","88,000.000","464,000.000","2,016.000","376,000.000","5,672.480","61,110,010,023,006.000",0.056,2017-01-01,Single Family Residential
2,14186244,2.000,3.000,"1,243.000","6,059.000","33,886,168.000","-117,823,170.000",122,261.000,"60,590,218.022","1,286.000","97,078.000",6.000,"1,962.000","85,289.000","564,778.000","2,016.000","479,489.000","6,488.300","60,590,218,022,012.000",0.005,2017-01-01,Single Family Residential
3,12177905,3.000,4.000,"2,376.000","6,037.000","34,245,180.000","-118,240,722.000",0101,261.000,"60,373,001.001","3,101.000","96,330.000",0.000,"1,970.000","108,918.000","145,143.000","2,016.000","36,225.000","1,777.510","60,373,001,001,006.000",-0.103,2017-01-01,Single Family Residential
4,10887214,3.000,3.000,"1,312.000","6,037.000","34,185,120.000","-118,414,640.000",010C,266.000,"60,371,236.012","3,101.000","96,451.000",0.000,"1,964.000","73,681.000","119,407.000","2,016.000","45,726.000","1,533.890","60,371,236,012,000.000",0.007,2017-01-01,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77569,10833991,3.000,3.000,"1,741.000","6,037.000","34,202,400.000","-118,502,000.000",010C,266.000,"60,371,319.001","3,101.000","96,415.000",0.000,"1,980.000","265,000.000","379,000.000","2,016.000","114,000.000","4,685.340","60,371,319,001,006.000",-0.002,2017-09-20,Condominium
77570,11000655,2.000,2.000,"1,286.000","6,037.000","34,245,368.000","-118,282,383.000",0100,261.000,"60,371,014.003","3,101.000","96,284.000",0.000,"1,940.000","70,917.000","354,621.000","2,016.000","283,704.000","4,478.430","60,371,014,003,002.000",0.021,2017-09-20,Single Family Residential
77571,17239384,2.000,4.000,"1,612.000","6,111.000","34,300,140.000","-118,706,327.000",1111,261.000,"61,110,084.022","2,061.000","97,116.000",7.000,"1,964.000","50,683.000","67,205.000","2,016.000","16,522.000","1,107.480","61,110,084,022,016.000",0.013,2017-09-21,Single Family Residential
77572,12773139,1.000,3.000,"1,032.000","6,037.000","34,040,895.000","-118,038,169.000",0100,261.000,"60,374,338.022","3,101.000","96,480.000",0.000,"1,954.000","32,797.000","49,546.000","2,016.000","16,749.000",876.430,"60,374,338,022,005.000",0.037,2017-09-21,Single Family Residential


In [21]:
def nulls_by_col(df):
    num_rows_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_miss = num_rows_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_rows_missing, 'percent_rows_missing': pct_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [22]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclasstypeid,77559,99.981
buildingclassdesc,77559,99.981
finishedsquarefeet13,77532,99.946
basementsqft,77524,99.936
storydesc,77524,99.936
storytypeid,77524,99.936
yardbuildingsqft26,77504,99.910
fireplaceflag,77402,99.778
architecturalstyletypeid,77368,99.734
architecturalstyledesc,77368,99.734


***

## Prepare

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [26]:
df.propertylandusetypeid.value_counts()

261.000    52438
266.000    19341
246.000     2021
269.000     1947
248.000      729
247.000      539
265.000      335
263.000       74
275.000       59
260.000       41
267.000       29
 31.000       15
264.000        6
Name: propertylandusetypeid, dtype: int64

### 2. Create a function that will drop rows or columns based on the percent of values that are missing: 

`handle_missing_values(df, prop_required_column, prop_required_row)`.
#### The input:
- A dataframe
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
#### The output:
- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.


In [ ]:
# want 60% -> pct_missing <= 40
# want 75% -> pct_missing <= 25

 & (missing.pct_cols_missing<=(1-prop_required_row))

In [58]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    # by columns
    num_rows_missing = df.isnull().sum()
    pct_rows_missing = num_rows_missing / df.shape[0] * 100
    # by rows
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = num_cols_missing / df.shape[1] * 100
    
    missing = pd.DataFrame({'pct_rows_missing': pct_rows_missing, 'pct_cols_missing': pct_cols_missing})
    columns = missing[(missing.pct_rows_missing<=(1-prop_required_column))].index
    print(columns)

In [59]:
newdf=handle_missing_values(df,0.6,0.6)

Index(['assessmentyear', 'bathroomcnt', 'bedroomcnt',
       'calculatedfinishedsquarefeet', 'censustractandblock', 'fips',
       'landtaxvaluedollarcnt', 'latitude', 'logerror', 'longitude',
       'parcelid', 'propertycountylandusecode', 'propertylandusedesc',
       'propertylandusetypeid', 'rawcensustractandblock', 'regionidcounty',
       'regionidzip', 'roomcnt', 'structuretaxvaluedollarcnt', 'taxamount',
       'taxvaluedollarcnt', 'transactiondate', 'yearbuilt'],
      dtype='object')


### 3. Encapsulate your work inside of functions in a wrangle_zillow.py module.